<a href="https://colab.research.google.com/github/DimaKav/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/DS_Unit_4_Sprint_3_Neural_Nets_Spring_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks Sprint Challenge

## 1) Define the following terms:

- Neuron

A single node in a neural network.

- Input Layer

The layer which typically corresponds to the number of dimensions in the dataset. Each unit of the input layer passes its assigned value to each neuron of the first hidden layer.

- Hidden Layer

Each hidden layer neuron multiplies each value passed in from the input layer with the weight, sums the multiplied values and applied activation function and passes this quantity to the layer, then this gets passed to the output layer.

- Output Layer

The final fully-connected layer in the network. It gives class scores in classification settings and others which are contextual to the model.


- Activation

When the neuron calculates a weighted sum it its input and adds a bias, the activation function decides whether the signal should be fired or not and to what extent. The activation function figures out the ideal threshold of this firing to allow the best matchin of inputs to the correct output class. 

- Backpropagation

After weights have been initialized, data fed in, dot products computed and passed through the activation function and errors have been calculated and this transmission of information is modified and made better through gradient descent -- backprop takes the error associated with a wrong guess and uses that error to ajust the NN's parameters in the direction of less error which is dictated by gradient descent. It essentially calculated gradient descent with respect to the weights. It's called backpropagation because the weights are updated from the (forward) first output.

## 2) Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 1  | 1  | 1  | 1 |
| 1  | 0  | 1  | 0 |
| 0  | 1  | 1  | 0 |
| 0  | 0  | 1  | 0 |

In [1]:
import numpy as np
np.random.seed(42)

inputs = np.array([
    [1, 1, 1],
    [1, 0, 1],
    [0, 1, 1],
    [0, 0, 1]
])

correct_outputs = np.array(([1], 
                            [0], 
                            [0], 
                            [0]))
correct_outputs

array([[1],
       [0],
       [0],
       [0]])

In [2]:
np.random.seed(42)
weights = 2 * np.random.random((3, 1)) - 1

for epoch in range(1000):
  # Functions to update the weights (sigmoid and its derivative)
  def sigmoid(x):
    return 1 / (1 + np.exp(-x))

  def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

  # Weighted sum of inputs and weights
  weighted_sum = np.dot(inputs, weights)

  # Activated value for the end of 1 training epoch
  activated_output = sigmoid(weighted_sum)

  # Calculate error by subtracting activated output from correct outputs
  error = correct_outputs - activated_output

  # Implement gradient descent and backpropagation
  adjustments = error * sigmoid_derivative(activated_output)

  # Update the weights with it
  weights += np.dot(inputs.T, adjustments)

print('Weights after training')
print(weights)

print('Output after training')
print(activated_output.round(2))

Weights after training
[[  7.20718058]
 [  7.20718153]
 [-11.09894178]]
Output after training
[[0.96]
 [0.02]
 [0.02]
 [0.  ]]


## 3) Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. 
- Your network must have one hidden layer. 
- You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
- Train your model on the Heart Disease dataset from UCI:

[Github Dataset](https://github.com/ryanleeallred/datasets/blob/master/heart.csv)

[Raw File on Github](https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv)


In [3]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [0]:
df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [0]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [0]:
df['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

In [0]:
from sklearn.preprocessing import StandardScaler
X = df.drop('target',axis=1).values
X = StandardScaler().fit_transform(X)
y = df['target'].values

In [5]:
X.shape, y.shape

((303, 13), (303,))

In [0]:
class Neural_Network(object):
  def __init__(self):
    self.inputs = 13
    self.hiddenNodes = 1
    self.outputNodes = 1

    # Initlize Weights
    self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes)
    self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes)

  def feed_forward(self, X):
    # Weighted sum between inputs and hidden layer
    self.hidden_sum = np.dot(X, self.L1_weights)
    # Activations of weighted sum
    self.activated_hidden = self.sigmoid(self.hidden_sum)
    # Weighted sum between hidden and output
    self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
    # final activation of output
    self.activated_output = self.sigmoid(self.output_sum)
    return self.activated_output
    
  def sigmoid(self, s):
    return 1/(1+np.exp(-s))
  
  def sigmoidPrime(self, s):
    return s * (1 - s)
  
  def backward(self, X, y, o):
    # backward propgate through the network
    self.o_error = y - o # error in output
    self.o_delta = self.o_error*self.sigmoidPrime(o) # applying derivative of sigmoid to error

    self.z2_error = self.o_delta.dot(self.L2_weights.T) # z2 error: how much our hidden layer weights contributed to output error
    self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden) # applying derivative of sigmoid to z2 error

    self.L1_weights += X.T.dot(self.z2_delta) # adjusting first set (input --> hidden) weights
    self.L2_weights += self.activated_hidden.T.dot(self.o_delta) # adjusting second set (hidden --> output) weights
    
  def train (self, X, y):
    o = self.feed_forward(X)
    self.backward(X, y, o)

In [0]:
NN = Neural_Network()
for i in range(100): # train the network 100 times
  if i+1 in [1,2,3,4,5] or (i+1) % 50 == 0:
    print("Input: \n", X) 
    print("Actual Output: \n", y)  
    print("Predicted Output: \n" + str(NN.feed_forward(X))) 
    print("Loss: \n" + str(np.mean(np.square(y - NN.feed_forward(X))))) # mean sum squared loss
    print("\n")
  NN.train(X, y)

In [6]:
from sklearn.neural_network import MLPClassifier

# Instantiate multilayer perceptron class (alternative implementation)
mlp = MLPClassifier(hidden_layer_sizes=(1,), activation='logistic', # one hidden layer
                    learning_rate_init=0.05, solver='sgd', max_iter=1000,
                    batch_size=20, momentum=0.5)
# Fit and score it
mlp.fit(X, y)
mlp.score(X, y)

0.8778877887788779

## 4) Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy. 

- Use the Heart Disease Dataset (binary classification)
- Use an appropriate loss function for a binary classification task
- Use an appropriate activation function on the final layer of your network. 
- Train your model using verbose output for ease of grading.
- Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
- When hyperparameter tuning, show you work by adding code cells for each new experiment. 
- Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
- You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [7]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=13, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='AdaGrad',
                metrics=['accuracy'])
	return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# Tune epochs
param_grid = {'epochs': [50, 100, 150]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/150
303/303 [==============================] - 0s 1ms/step - loss: 0.6353 - acc: 0.6370
Epoch 2/150
303/303 [==============================] - 0s 70us/step - loss: 0.5588 - acc: 0.7327
Epoch 3/150
303/303 [==============================] - 0s 59us/step - loss: 0.5237 - acc: 0.7591
Epoch 4/150
303/303 [==============================] - 0s 62us/step - loss: 0.4995 - acc: 0.7624
Epoch 5/150
303/303 [==============================] - 0s 60us/step - loss: 0.4811 - acc: 0.7657
Epoch 6/150
303/303 [==============================] - 0s 66us/step - loss: 0.4664 - acc: 0.7624
Epoch 7/150
303/303 [==============================] - 0s 64us/step - loss: 0.4542 - acc: 0.7690
Epoch 8/150
303/303 [==============================] - 0s 65us/step - loss: 0.4440 - acc: 0.7855
Epoch 9/150
303/303 [==============================] - 0s 63us/step - loss: 0.4348 - acc: 0.8053
Epoch 10/150
303/303 [==============================] - 0s 68us/step - loss: 0.4269 - acc: 0.8185
Epoch 11/150
303/303 [========

In [0]:
# Tune batch size

param_grid = {'epochs': [150],
              'batch_size': [30,60,90]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/150
303/303 [==============================] - 0s 1ms/step - loss: 0.7358 - acc: 0.5875
Epoch 2/150
303/303 [==============================] - 0s 41us/step - loss: 0.6354 - acc: 0.6535
Epoch 3/150
303/303 [==============================] - 0s 50us/step - loss: 0.5893 - acc: 0.6931
Epoch 4/150
303/303 [==============================] - 0s 52us/step - loss: 0.5621 - acc: 0.7096
Epoch 5/150
303/303 [==============================] - 0s 51us/step - loss: 0.5368 - acc: 0.7360
Epoch 6/150
303/303 [==============================] - 0s 40us/step - loss: 0.5209 - acc: 0.7426
Epoch 7/150
303/303 [==============================] - 0s 56us/step - loss: 0.5088 - acc: 0.7624
Epoch 8/150
303/303 [==============================] - 0s 39us/step - loss: 0.4990 - acc: 0.7723
Epoch 9/150
303/303 [==============================] - 0s 51us/step - loss: 0.4904 - acc: 0.7789
Epoch 10/150
303/303 [==============================] - 0s 41us/step - loss: 0.4847 - acc: 0.7756
Epoch 11/150
303/303 [========

In [8]:
# Tune init, optimizers, and loss function

def create_model(optimizer='rmsprop', init='glorot_uniform'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=13, kernel_initializer=init, activation='relu'))
	model.add(Dense(8, kernel_initializer=init, activation='relu'))
	model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
model = KerasClassifier(build_fn=create_model, verbose=0)

init = ['uniform', 'glorot_uniform']
optimizers = ['AdaGrad', 'sgd', 'Adam']
loss = ['binary_crossentroy', 'sigmoid']
epochs = [150]
batches = [60]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, y)

# results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.745875 using {'batch_size': 60, 'epochs': 150, 'init': 'uniform', 'optimizer': 'AdaGrad'}
0.745875 (0.060676) with: {'batch_size': 60, 'epochs': 150, 'init': 'uniform', 'optimizer': 'AdaGrad'}
0.122112 (0.172693) with: {'batch_size': 60, 'epochs': 150, 'init': 'uniform', 'optimizer': 'sgd'}
0.702970 (0.077118) with: {'batch_size': 60, 'epochs': 150, 'init': 'uniform', 'optimizer': 'Adam'}
0.646865 (0.111530) with: {'batch_size': 60, 'epochs': 150, 'init': 'glorot_uniform', 'optimizer': 'AdaGrad'}
0.603960 (0.106022) with: {'batch_size': 60, 'epochs': 150, 'init': 'glorot_uniform', 'optimizer': 'sgd'}
0.653465 (0.124454) with: {'batch_size': 60, 'epochs': 150, 'init': 'glorot_uniform', 'optimizer': 'Adam'}
